# Utilities for Type 1a Supernova Fits
 >__Created__:  Summer 2017 Harrison B. Prosper<br>
 >__Updated__:  Fall 2021 for LPC Stats 2021


In [1]:
# standard system modules
import os, sys

# standard table manipulation module
#import pandas as pd

# standard array manipulation module
import numpy as np
from array import array

# standard scientific python module
#import scipy as sp
#import scipy.stats as st
#import scipy.optimize as op

# CERN data analysis package
import ROOT

# wrapper around Minuit
import iminuit as im

# standard symbolic algebra pakage
#import sympy as sm

# standard plotting module
import matplotlib as mp
import matplotlib.pyplot as plt

# make plots appear inline
%matplotlib inline

Welcome to JupyROOT 6.24/00


In [2]:
# update fonts
font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 20
        }
mp.rc('font', **font)
mp.rc('xtick', labelsize='x-small')
mp.rc('ytick', labelsize='x-small')

# set usetex = False if Latex is not available on your system
mp.rc('text', usetex=True)

### Model parameters

  * ID: model identifier
  * free: specifies whether parameter is free
  * name: name of parameter
  * guess: starting (or fixed) value of parameter

In [3]:
LCDM  = 0
PHANTOM = 1

#                      ID,     (name,  guess, min, max)
PARAMS = {'LCDM' :   [LCDM,    [('OM', 0.3, 0.0, 10.0), 
                                ('OL', 0.7, 0.0, 10.0), 
                                ('H0', 70,  1.0,200.0)]],
          'phantom': [PHANTOM, [('n',   2,  0.0, 10.0), 
                                ('H0', 70,  1.0,200.0)]]           
         }

# define ranges for redshifts and distance moduli
ZMIN  = 0.0 
ZMAX  = 1.6
MUMIN = 32.0
MUMAX = 46.0

# bag for data etc.
class Bag: pass

### Compile C++ classe CosmicCode using ROOT

In [4]:
def compileCode(bag, modelparams):
    from ROOT import gROOT

    gROOT.ProcessLine(open('../CosmicCode.cc').read())
    from ROOT import CosmicCode
    
    # make sure model name is valid
    if not (bag.name in modelparams):
        print("** unknown model %s" % bag.name)
    
    # get model id and parameters and code
    ID, params = modelparams[bag.name]
    code       = CosmicCode(ID)
    return (code, params)

### Read Type 1a supernova data 

In [5]:
def read_data(filename):
    # Skip first 5 rows then read columns 1, 2, and 3
    z, mu, dmu = np.loadtxt(filename, 
                            delimiter='\t', 
                            skiprows=5, 
                            usecols=(1,2,3), 
                            unpack=True)

    print("number of observations: %d" % len(z))
    print("%5s\t%10s\t%10s +/- %-10s" % ('', 'z', 'x', 'dx'))
    for ii, (u, x, dx) in enumerate(zip(z, mu, dmu)):
        if ii % 100 == 0:
            print("%5d\t%10.3f\t%10.4f +/- %-10.4f"% (ii, u, x, dx))
            
    return (z, mu, dmu)

### Simple class to annotate plots

In [6]:
class Scribe:
    def __init__(self, xpos, lineno, nlines=12, ftsize=16):
        
        self.ftsize = ftsize
        
        axes = plt.gca()
        self.xmin, self.xmax = axes.get_xlim()
        self.ymin, self.ymax = axes.get_ylim()
        self.ystep = (self.ymax-self.ymin) / nlines
        self.xpos  = xpos
        self.ypos  = self.ymax - self.ystep * lineno
        
    def __def__(self):
        pass
    
    def write(self, line, indent=0):
        plt.text(self.xpos+indent, self.ypos, line, 
                 fontsize=self.ftsize)
        self.ypos -= self.ystep

### Annotation for cosmological models

In [7]:
def annotate(bag, scribe, offset=0.2):
    if bag.name == 'LCDM':
        OM = (bag.x[0], np.sqrt(bag.cov[0][0]))
        OL = (bag.x[1], np.sqrt(bag.cov[1][1]))

        scribe.write(r"$\Lambda CDM$ model")
        scribe.write("")
        scribe.write(r"$\Omega(a) = \frac{\Omega_{M}}{a^{3}} + "\
                     r"\frac{(1 - \Omega_{M} - "\
                     r"\Omega_{\Lambda})}{a^{2}}"\
                     r" + \Omega_{\Lambda}$", 
                     offset)
        scribe.write(" ")
        scribe.write(r"$\Omega_{M} = %5.2f \pm %-5.2f$" % OM, offset)
        scribe.write(r"$\Omega_{\Lambda} = %5.2f \pm %-5.2f$" % OL,
                      offset)
    else:
        n, dn = (bag.x[0], np.sqrt(bag.cov[0][0]))
        x = 3.0/(2*n)
        G = sp.special.gamma(x)
        T = G*np.sqrt(np.e)*2**x/n        
        scribe.write("phantom model")
        scribe.write("")
        scribe.write(r"$\Omega(a) = \frac{\Omega_{M}}{a^{3}} + "\
                     r"\frac{e^{a^{n}-1} - \Omega_{M}}{a^{3}}$", 
                     offset)
        scribe.write("")
        scribe.write(r"$H_{0}t = \sqrt{e} 2^{3/(2n)} "\
                     r"\Gamma(3/(2n), a^{n}/2)/n$", 
                     offset)
        scribe.write("")
        scribe.write(r"$H_{0}t_{rip} = \sqrt{e} 2^{3/(2n)}"\
                     r" \Gamma(3/(2n))/n = %4.2f$" % T, 
                     offset)
        scribe.write("")
        scribe.write(r"where $\Gamma(s, x) = "\
                     r"\int_{0}^{x} t^{s-1} e^{-t} dt$", 
                     offset)
        scribe.write(r"and $n = %4.2f \pm %-4.2f$" % (n, dn), 
                     offset)    

### Plot data and optional superimpose fit

In [8]:
def plot_data(bag, nll=None,
              zmin=ZMIN, zmax=ZMAX, 
              mumin= MUMIN, mumax=MUMAX, 
              ftsize=16, 
              fgsize=(8, 5)):
  
    # set size of figure
    plt.figure(figsize=fgsize)
    
    plt.errorbar(bag.z, bag.mu, yerr=bag.dmu, 
                 fmt='o', 
                 ecolor='steelblue', markersize=2,
                 color='blue', label='data')
        
    # set up x, y limits
    plt.xlim(zmin, zmax)
    plt.ylim(mumin, mumax)
   
    # add x and y labels
    plt.xlabel('$z$', fontsize=20)
    plt.ylabel('$\mu$', fontsize=20)
    
    # annotate 
    xwid = (zmax-zmin)/10
    xpos = zmin  + 3*xwid
    
    scribe = Scribe(xpos, lineno=6)
    scribe.write('The Union2.1 Compilation')     
    scribe.write('The Supernova Cosmology Project')
       
    if nll != None:
        filename = 'fig_' + bag.name + '_union_2_1_fit.pdf'
        
        # name:  name of cosmological model
        # x:   fitted parameters
        # p:   parameters to be passed to distanceModulus
        chi2 = 2 * nll(bag.x, bag)
                
        ndf  = len(bag.z) - len(bag.x) # number of degrees of freedom
    
        # compute best-fit model
        nz   = 100
        zstep= (zmax - zmin) / nz
        zz   = np.arange(zmin+zstep/2, zmax, zstep)
        ff   = [bag.code.distanceModulus(u, bag.p) for u in zz]

        plt.plot(zz, ff, color='red', label='%s model' % bag.name)
        
        scribe.write(r"$\chi^{2} / {\rm ndf} = %5.1f / %d = %5.2f$"%\
                     (chi2, ndf, chi2/ndf))
        
        plt.legend()
    else:
        filename = "fig_union_2_1_data.pdf"
    
    # tighten layout so that image is fully
    # contained within viewport
    
    plt.tight_layout()
    
    print('\n%s' % filename)
    plt.savefig(filename)
    plt.show()

### Plot predicted scale factor vs. time

In [9]:
def plot_scale_factor(bag, 
                      tmin=0, tmax=1.6, 
                      amin=0, amax=10, 
                      ftsize=16, 
                      fgsize=(6, 5)):
  
    # set size of figure
    plt.figure(figsize=fgsize)
    
    # set up x, y limits
    plt.xlim(tmin, tmax)
    plt.ylim(amin, amax)
   
    # add x and y labels
    plt.xlabel('$H_0 t$', fontsize=20)
    plt.ylabel('$a(t)$', fontsize=20)
    
    # plot a(x) vs x = H0*t plot
    a = array('d'); a.fromlist(bag.code.N*[0])
    t = array('d'); t.fromlist(bag.code.N*[0])
    bag.code.scaleFactor(amax, bag.p, t, a)
    
    plt.plot(t, a, color='blue')
       
    # plot horizontal line at a = 1
    plt.plot([tmin, tmax], [1, 1], color='magenta')
    
    # annotate 
    xwid = (tmax-tmin)/12
    xpos = tmin + xwid/2
    offset = 0.5*xwid
    scribe = Scribe(xpos, lineno=1, nlines=12)
    annotate(bag, scribe, offset)
    
    # tighten layout so that image is fully
    # contained within viewport
    
    plt.tight_layout()
    
    filename = 'fig_' + bag.name + '_scale_factor.pdf'
    print('\n%s' % filename)
    plt.savefig(filename)
    
    plt.show()